In [1]:
from google import genai
import json
from datetime import datetime
from html import escape

In [ ]:
with open("./data/reviews.json", "r", encoding="utf-8") as f:
    data = json.load(f)
client = genai.Client(api_key='YourKey')
chunk_size = 13
##chunk process for large number of reviews
def chunk_process(chunk,data,prompt):
    processed_chunk = []
    for i in range(0,len(data),chunk):
        elements = data[i:i+chunk]
        review_text = '\n'.join(
            f'{idx+1},{x}' for idx,x in enumerate(elements)
        )
        response = client.models.generate_content(
        model="gemini-3-flash-preview", contents=prompt+review_text
    )       
        processed_chunk.append(response.text)
        
    return processed_chunk
#aggregate processed chunks and output report in json format
def aggregate(processed_content,prompt):
    review_processed = '\n'.join(
            x for idx,x in enumerate(processed_content)
        )
    response = client.models.generate_content(
        model="gemini-3-flash-preview", contents=prompt+review_processed
    )  
    return response.text
def json_to_html_report(data: dict) -> str:
    meta = data.get("meta", {})
    total = meta.get("total_samples_analyzed", 0)
    generated_at = meta.get("generated_at") or datetime.now().strftime("%Y-%m-%d")

    def severity_color(percentage_str: str) -> str:
        # 按百分比映射颜色：>=50 红，>=30 橙，其它黄
        try:
            p = int(percentage_str.replace("%", "").strip())
        except Exception:
            p = 0
        if p >= 50:
            return "pill red"
        if p >= 30:
            return "pill orange"
        return "pill yellow"

    parts = []
    parts.append(f"""<!doctype html>
<html lang="zh-CN">
<head>
<meta charset="utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>AI 差评梳理</title>
<style>
  :root {{
    --bg: #ffffff;
    --text: #111827;
    --muted: #6b7280;
    --card: #f9fafb;
    --line: #e5e7eb;
    --quote-bg: #fff1f0;
    --quote-line: #ffccc7;
    --red: #ff4d4f;
    --orange: #fa8c16;
    --yellow: #fadb14;
  }}

  body {{
    margin: 0;
    background: var(--bg);
    color: var(--text);
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "PingFang SC",
                 "Hiragino Sans GB", "Microsoft YaHei", "Noto Sans CJK SC", Arial, sans-serif;
  }}

  .container {{
    max-width: 720px;
    margin: 0 auto;
    padding: 14px 14px 40px;
  }}

  .header {{
    background: #f3f4f6;
    border: 1px solid var(--line);
    border-radius: 14px;
    padding: 14px 14px;
    margin-bottom: 14px;
  }}
  .header h1 {{
    margin: 0;
    font-size: 18px;
    line-height: 1.2;
  }}
  .header p {{
    margin: 6px 0 0;
    color: var(--muted);
    font-size: 13px;
  }}

  .category {{
    margin-top: 14px;
  }}
  .category-title {{
    font-size: 16px;
    font-weight: 700;
    margin: 12px 2px 8px;
  }}

  details {{
    background: #fff;
    border: 1px solid var(--line);
    border-radius: 12px;
    margin: 8px 0;
    overflow: hidden;
  }}

  summary {{
    list-style: none;
    cursor: pointer;
    padding: 12px 12px;
    display: flex;
    align-items: center;
    justify-content: space-between;
    gap: 10px;
  }}
  summary::-webkit-details-marker {{ display: none; }}

  .left {{
    display: flex;
    align-items: center;
    gap: 10px;
    min-width: 0;
  }}

  .tag {{
    font-weight: 700;
    font-size: 15px;
    white-space: nowrap;
    overflow: hidden;
    text-overflow: ellipsis;
  }}

  .right {{
    display: flex;
    align-items: center;
    gap: 8px;
    flex-shrink: 0;
  }}

  .pill {{
    font-size: 12px;
    padding: 4px 8px;
    border-radius: 999px;
    color: #fff;
    font-weight: 700;
  }}
  .pill.red {{ background: var(--red); }}
  .pill.orange {{ background: var(--orange); }}
  .pill.yellow {{ background: #9ca3af; }}

  .arrow {{
    color: var(--muted);
    font-size: 14px;
  }}

  .content {{
    border-top: 1px solid var(--line);
    padding: 12px 12px 14px;
  }}

  .desc {{
    color: var(--muted);
    font-size: 13px;
    line-height: 1.5;
    margin: 0 0 10px;
  }}

  .quote-box {{
    background: var(--quote-bg);
    border: 1px solid var(--quote-line);
    border-radius: 10px;
    padding: 10px;
  }}
  .quote {{
    margin: 6px 0;
    font-size: 13px;
    line-height: 1.45;
  }}

  .meta {{
    margin-top: 6px;
    color: var(--muted);
    font-size: 12px;
  }}
</style>
</head>
<body>
  <div class="container">
    <div class="header">
      <h1>🛡️ AI 差评梳理</h1>
      <p>基于近 30 天 {escape(str(total))} 条低分评价 · 生成日期 {escape(generated_at)}</p>
    </div>
""")

    for cat in data.get("categories", []):
        cat_name = escape(cat.get("name", ""))
        parts.append(f'<div class="category"><div class="category-title">{cat_name}</div>')

        for item in cat.get("items", []):
            tag = escape(item.get("tag", ""))
            desc = escape(item.get("description", ""))
            pct = escape(item.get("stats", {}).get("percentage", "0%"))
            mention_count = item.get("stats", {}).get("mention_count", 0)

            pill_class = severity_color(pct)

            quotes = item.get("evidence_quotes", []) or []
            quote_html = "\n".join(
                f'<div class="quote">💬 “{escape(str(q))}”</div>' for q in quotes
            )

            parts.append(f"""
<details>
  <summary>
    <div class="left">
      <div class="tag">{tag}</div>
    </div>
    <div class="right">
      <div class="{pill_class}">{pct} 提及</div>
      <div class="arrow">⌄</div>
    </div>
  </summary>
  <div class="content">
    <p class="desc">📝 {desc}</p>
    <div class="quote-box">
      {quote_html if quote_html else '<div class="quote">（暂无可展示的原文摘录）</div>'}
    </div>
    <div class="meta">提及次数：{escape(str(mention_count))}</div>
  </div>
</details>
""")

        parts.append("</div>")  # end category

    parts.append("""
  </div>
<script>
  // 让箭头跟随展开状态变化（纯装饰）
  document.querySelectorAll("details").forEach(d => {
    const arrow = d.querySelector(".arrow");
    const update = () => { arrow.textContent = d.open ? "⌃" : "⌄"; };
    d.addEventListener("toggle", update);
    update();
  });
</script>
</body>
</html>
""")
    return "".join(parts)

prompt_chunk = f""" 
你是一个客观的差评数据清洗员。
任务：阅读输入的【差评列表】，提取具体的负面槽点，并按维度分类。
要求：
1. **维度分类**：分为 [食材/口味]、[服务]、[环境/卫生]、[性价比/其他]。
2. **语义聚合**：在同一批次中，如果多人提到“肉硬”、“咬不动”，归纳为一个槽点“肉质老/咬不动”。
3. **统计频次**：统计该槽点在本批次中出现的次数。
4. **保留证据**：摘录 1 句最能代表该槽点的原话（越短越好）。
5. **输出格式**：JSON 格式，不要任何废话。
输出格式示例：
{{
  "total_reviews_in_batch": 输入评论的数量,
  "issues": [
    {{
      "category": "食材/口味",
      "topic": "肉质口感差",
      "count": 1,
      "key_evidence": "肉比鞋底还硬，咬肌都累了"
    }},
    {{
      "category": "服务",
      "topic": "晚间补货不及时",
      "count": 2,
      "key_evidence": "7点半以后就不补菜了，吃了个寂寞"
    }}
  ]
}}
6.输入数据（评论）如下：

"""
prompt_aggregate = """ 
# Role
你是一个智能数据整合系统。你的唯一任务是接收多份差评分析片段，合并并输出一份标准的 JSON 格式报告。

# Constraints
1. **绝对客观**：在生成 `tag` 和 `description` 时，必须使用中立、陈述性的语言。
   - 🚫 禁用词汇：恶心、垃圾、令人发指、黑心商家。
   - ✅ 推荐词汇：口感不佳、异物、补货慢、服务响应迟缓。
2. **禁止废话**：输出内容必须且只能是 JSON 字符串，不要包含 Markdown 标记（如 ```json），不要包含任何开场白或结束语。
3. **原文引用**：`quotes` 字段必须摘录用户的原始评论，不做任何修改。
4. **数据计算**：
   - `mention_count`：即使不同批次提及了同一个问题，也要将数量累加。
   - `percentage`：基于提供的总样本数计算百分比，保留百分号。

# JSON Output Format (Strict Adherence)
请严格按照以下 JSON 结构输出结果。不要修改字段名称（Key），仅填充 Value：

{
  "meta": {
    "total_samples_analyzed": <Int, 本次分析的总评论数>,
    "generated_at": "<String, 当前日期>"
  },
  "categories": [
    {
      "name": "<String, 维度名称，固定为：食材口味 | 服务运营 | 环境卫生 | 性价比/其他>",
      "items": [
        {
          "tag": "<String, 3-6个字的短标签，如：肉质口感硬>",
          "description": "<String, 客观描述，如：多名用户反馈肉质咬不动，疑似合成肉>",
          "stats": {
            "mention_count": <Int, 提及次数>,
            "percentage": "<String, 占比，如 45%>"
          },
          "evidence_quotes": [
            "<String, 原始评论摘录1>",
            "<String, 原始评论摘录2>"
          ]
        }
      ]
    }
  ]
}

# Input Data
(此处等待接收分批处理后的 JSON 数据流...)

"""

In [3]:
def main():
    processed_content = chunk_process(chunk_size,data=data,prompt=prompt_chunk)
    report = aggregate(processed_content=processed_content,prompt=prompt_aggregate)
    report_json = json.loads(report)
    html = json_to_html_report(report_json)
    with open("report.html", "w", encoding="utf-8") as f:
        f.write(html)
main()

In [5]:
len(data)

25